In [1]:
!ollama ls

NAME                    	ID          	SIZE  	MODIFIED          
llama3:8b               	365c0bd3c000	4.7 GB	About an hour ago	
llava:latest            	8dd30f6b0cb1	4.7 GB	40 hours ago     	
llama3:8b-instruct-q5_1 	662158bc9277	6.1 GB	11 days ago      	
mistral:latest          	2ae6f6dd7a3d	4.1 GB	12 days ago      	
llama3-gradient:latest  	5d1398df5b8b	4.7 GB	2 weeks ago      	
mxbai-embed-large:latest	468836162de7	669 MB	2 weeks ago      	
llama3:latest           	365c0bd3c000	4.7 GB	2 weeks ago      	
llama2:latest           	78e26419b446	3.8 GB	2 weeks ago      	
phi3:3.8b               	64c1188f2485	2.4 GB	2 weeks ago      	


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
from utilities.ollamaAdditionalFunctions import CustomOllamaFunctions, get_ollama_tools_system_message, get_ollama_custom_chatPrompt, create_ollama_functions_agent

# tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.tools import Tool, StructuredTool, tool

# getting LLM
llm_ollama_func = CustomOllamaFunctions(model="llama3:8b", format="json")
# llm_ollama_func = CustomOllamaFunctions(model="mistral", format="json")

# create/get some tools
# tools = [TavilySearchResults(max_results=1), 
#          ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)), 
#          WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500))]

@tool
def google_search(query: str):
    """Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.

    Args:
        query (str): Query to search on Google.

    Returns:
        str: Google search results.
    """
    search = GoogleSearchAPIWrapper(k=1)
    return search.run(query)

tools = [google_search]

# getting system message
system_message = get_ollama_tools_system_message(tools)

# getting chat prompt
prompt = get_ollama_custom_chatPrompt()

# creating agent
agent = create_ollama_functions_agent(llm_ollama_func, prompt)
agent

functions: [{'name': 'google_search', 'parameters': {'title': 'google_searchSchema', 'description': 'Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\n\nArgs:\n    query (str): Query to search on Google.\n\nReturns:\n    str: Google search results.', 'type': 'object', 'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query']}, 'description': 'Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\n\n    Args:\n        query (str): Query to search on Google.\n\n    Returns:\n        str: Google search results.'}, {'name': 'conversational_response', 'description': 'Respond conversationally if no other tools should be called for a given query.', 'parameters': {'type': 'object', 'properties': {'response': {'type': 'string', 'description': 'Conversational response to the user.'}}, 'required': ['response']}}]
sys mess: content='You are a helpful assistant. 

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_ollama_tool_messages(x['intermediate_steps'], last_n_messages=last_n_messages))
})
| RunnableAssign(mapper={
    print_input: RunnableLambda(lambda x: print(f'Agent Input - Inside agent chain: {x}'))
  })
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'system_message'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_

### Langgraph components

#### StateGraph

In [4]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    # The input string
    input: str
    # System message
    system_message: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

#### Defining functions for Agent Graph to be built

In [5]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)

# Define the agent
def run_agent(data):
    print(f"keys in run agent: {data.keys()}")
    agent_outcome = agent.invoke(data)
    print("Agent Outcome: ", agent_outcome)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    print(f"keys in run tools: {data.keys()}, {len(data["intermediate_steps"])}")
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    print(f"keys in run continue: {data.keys()}, {len(data["intermediate_steps"])}")
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

#### AgentGraph / LangGraph

In [6]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

#### Calling AgentGraph

In [7]:
# Ollama - Llama3:8b + Google Search
# inputs = {"input": "what is the weather in sf", "chat_history": [], "system_message": str(system_message.content)}
inputs = {"input": "what is machine learning in simple words?", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "Find todays weather in san francisco?", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "Who is the current Prime Minister of Australia?", "chat_history": [], "system_message": str(system_message.content)}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

keys in run agent: dict_keys(['input', 'system_message', 'chat_history', 'agent_outcome', 'intermediate_steps'])
Agent Input - Inside agent chain: {'input': 'what is machine learning in simple words?', 'system_message': 'You are a helpful assistant. You have access to the following tools:\n\n[\n  {\n    "name": "google_search",\n    "parameters": {\n      "title": "google_searchSchema",\n      "description": "Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\\n\\nArgs:\\n    query (str): Query to search on Google.\\n\\nReturns:\\n    str: Google search results.",\n      "type": "object",\n      "properties": {\n        "query": {\n          "title": "Query",\n          "type": "string"\n        }\n      },\n      "required": [\n        "query"\n      ]\n    },\n    "description": "Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\\n\\n    Args:\\n        query (str): Query 

In [7]:
# Ollama - Llama3:8b + Google Search
# inputs = {"input": "what is the weather in sf", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "what is machine learning in simple words?", "chat_history": [], "system_message": str(system_message.content)}
inputs = {"input": "Find todays weather in san francisco?", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "Who is the current Prime Minister of Australia?", "chat_history": [], "system_message": str(system_message.content)}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

keys in run agent: dict_keys(['input', 'system_message', 'chat_history', 'agent_outcome', 'intermediate_steps'])
Agent Input - Inside agent chain: {'input': 'Find todays weather in san francisco?', 'system_message': 'You are a helpful assistant. You have access to the following tools:\n\n[\n  {\n    "name": "google_search",\n    "parameters": {\n      "title": "google_searchSchema",\n      "description": "Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\\n\\nArgs:\\n    query (str): Query to search on Google.\\n\\nReturns:\\n    str: Google search results.",\n      "type": "object",\n      "properties": {\n        "query": {\n          "title": "Query",\n          "type": "string"\n        }\n      },\n      "required": [\n        "query"\n      ]\n    },\n    "description": "Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\\n\\n    Args:\\n        query (str): Query to s

In [6]:
# Ollama - Llama3:8b + Google Search
# inputs = {"input": "what is the weather in sf", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "what is machine learning in simple words?", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "Find todays weather in san francisco?", "chat_history": [], "system_message": str(system_message.content)}
inputs = {"input": "Who is the current Prime Minister of Australia?", "chat_history": [], "system_message": str(system_message.content)}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

keys in run agent: dict_keys(['input', 'system_message', 'chat_history', 'agent_outcome', 'intermediate_steps'])
Agent Input - Inside agent chain: {'input': 'Who is the current Prime Minister of Australia?', 'system_message': 'You are a helpful assistant. You have access to the following tools:\n\n[\n  {\n    "name": "google_search",\n    "parameters": {\n      "title": "google_searchSchema",\n      "description": "Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\\n\\nArgs:\\n    query (str): Query to search on Google.\\n\\nReturns:\\n    str: Google search results.",\n      "type": "object",\n      "properties": {\n        "query": {\n          "title": "Query",\n          "type": "string"\n        }\n      },\n      "required": [\n        "query"\n      ]\n    },\n    "description": "Search Google for recent results. For queries on recent topics/news/updates/weather/general_knowledge etc.\\n\\n    Args:\\n        query (str): 

In [8]:
# Ollama - Llama3:8b + TavilySearchResults
# inputs = {"input": "what is the weather in sf", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "what is machine learning in simple words?", "chat_history": [], "system_message": str(system_message.content)}
# inputs = {"input": "Find todays weather in san francisco?", "chat_history": [], "system_message": str(system_message.content)}
inputs = {"input": "Who is the current Prime Minister of Australia?", "chat_history": [], "system_message": str(system_message.content)}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

keys in run agent: dict_keys(['input', 'system_message', 'chat_history', 'agent_outcome', 'intermediate_steps'])
Agent Input - Inside agent chain: {'input': 'Who is the current Prime Minister of Australia?', 'system_message': 'You are a helpful assistant. You have access to the following tools:\n\n[\n  {\n    "name": "tavily_search_results_json",\n    "parameters": {\n      "title": "TavilyInput",\n      "description": "Input for the Tavily tool.",\n      "type": "object",\n      "properties": {\n        "query": {\n          "title": "Query",\n          "description": "search query to look up",\n          "type": "string"\n        }\n      },\n      "required": [\n        "query"\n      ]\n    },\n    "description": "A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query."\n  },\n  {\n    "name": "conversational_response",\n    "description": "Respond conversationally